<h1>Without the fancy charts</h1><br>
<h2>Load the data</h2>
The table shows the top 5 rows...


In [2]:
import pandas as pd
def loadData(csvLoc): 
    return pd.read_csv(csvLoc)
dSet=loadData("housing.csv")
dSet.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


...and a quick overview for sanity purposes


In [3]:
dSet.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


<h2> Split the data </h1>
We are making a stratified random split for this


In [4]:
import numpy as numpty
from sklearn.model_selection import StratifiedShuffleSplit

def chopper (data, testPortion):
    
    #make a categoricl attribute out of median income
    data["income_cat"]= pd.cut(data["median_income"],bins=[0., 1.5, 3.0, 4.5, 6., numpty.inf], labels=[1, 2, 3, 4, 5])
    
    #make arrays of shuffled indices from the dataset that both have the same distribution
    split=StratifiedShuffleSplit(n_splits=1,test_size=testPortion, random_state=42)
    
    #add the rows from each array to the respective output arrays
    for train_index, test_index in split.split(data, data["income_cat"]):
        testSet=data.loc[test_index]
        trainSet=data.loc[train_index]
        
     #remove the categoric attribute we created earlier   
    for set_ in (testSet,trainSet):
        set_.drop("income_cat",axis=1,inplace=True)        
    #victory!
    return testSet, trainSet    
            
strat_test_set, strat_train_set = chopper(dSet,.2)


overview of the training set - looks right to me...

In [5]:
strat_train_set.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 17606 to 15775
Data columns (total 10 columns):
longitude             16512 non-null float64
latitude              16512 non-null float64
housing_median_age    16512 non-null float64
total_rooms           16512 non-null float64
total_bedrooms        16354 non-null float64
population            16512 non-null float64
households            16512 non-null float64
median_income         16512 non-null float64
median_house_value    16512 non-null float64
ocean_proximity       16512 non-null object
dtypes: float64(9), object(1)
memory usage: 1.4+ MB


cleaning data

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import  Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

#drop the median house value and copy it into a set of labels before we start cleaning data
housing=strat_train_set.drop("median_house_value", axis=1)
housing_labels=strat_train_set["median_house_value"].copy()

#make a Transformer class to add some more attributes
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

#rooms_ix,bedrooms_ix, population_ix, household_ix=3,4,5,6
# get the right column indices: safer than hard-coding indices 3, 4, 5, 6
rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]


class CombinedAttributesAdder(BaseEstimator, TransformerMixin):

    def __init__(self,add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room

    def fit(self,X,y=None):
        return self

    def transform(self, X, y=None):
        #claculate the extra attributes
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        #claculate the optional one if needed
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
import bookeyWookey as book        

#set up a pipeline to process the numeric attributes using the new class and others    
num_pipe = Pipeline([('imputer',SimpleImputer(strategy="median")),
                    ('attribs_adder',book.CombinedAttributesAdder()),
                    ('std_scaler',StandardScaler()),                    
                    ])

#split off the numeric attributes...
housingNumeric = housing.drop("ocean_proximity", axis=1)
#...Store the column headings
numeric=list(housingNumeric)
#...same for the categorical attributes (1 of them)
categoric=["ocean_proximity"]


#set up the pipeline to output the complete transformation
full_pipe = ColumnTransformer([
    ("num",num_pipe,numeric),
    ("cat",OneHotEncoder(),categoric),
    ])
#execute the pipeline to produce prepared training data
trainingDataPrepped=full_pipe.fit_transform(housing)
trainingDataPrepped.shape

(16512, 16)

now to fit a model


In [10]:
from sklearn.linear_model import LinearRegression
type(housing_labels)
type(trainingDataPrepped)
trainingDataPrepped 
linReg=LinearRegression()
linReg.fit(trainingDataPrepped,housing_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

OK we has dun a AI!<br>
now how accurate is it?

In [11]:
from sklearn.metrics import mean_squared_error
exampleData = housing.iloc[:5]
exampleLabels = housing_labels.iloc[:5]
preppedExamples=full_pipe.transform(exampleData)
print("predictions: ",linReg.predict(preppedExamples))
print("Lables: ",list(exampleLabels))



predictions = linReg.predict(trainingDataPrepped)
linRegMSE = mean_squared_error(housing_labels,predictions)
linRegRMSE=np.sqrt(linRegMSE)
linRegMSE


predictions:  [210644.60459286 317768.80697211 210956.43331178  59218.98886849
 189747.55849879]
Lables:  [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


4709829587.971121

not brilliant, but better than the book!? <br>
Lets try a decision tree...

In [12]:
from sklearn.tree import DecisionTreeRegressor 
dtReg=DecisionTreeRegressor()
dtReg.fit(trainingDataPrepped,housing_labels)


dtPredictions=dtReg.predict(trainingDataPrepped)
dtMSE=mean_squared_error(housing_labels,dtPredictions)
dtRMSE=np.sqrt(dtMSE)
dtRMSE

0.0

Bollocks! There is no way it is perfect! There is a whiff of over fitting here<br/>
Let's do some cross-validation, i.e. splitting the training set up into smaller training and test sets. 
We are using k-fold validation which splits the traing data into 10 subsets, using each one in turn for validation and training on the other 9


In [13]:
from sklearn.model_selection import cross_val_score
#gets an array of scores mean squared errors
scores = cross_val_score(dtReg, trainingDataPrepped, housing_labels, scoring="neg_mean_squared_error", cv=10)
#Scikit-Learn’s cross-validation features expect a utility function (greater is better) rather than a cost function (lower is better), 
# so the scoring function is actually the opposite of the MSE (i.e., a negative value), we compute -scores before calculating the square root.
tree_rmse_scores = np.sqrt(-scores)

def getYerScoresOut(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
getYerScoresOut(tree_rmse_scores)

Scores: [68347.56445098 66908.28494553 71286.78319679 70196.89598448
 71104.82820956 75841.10965079 71297.04508401 71225.80468213
 77644.92515436 68428.3239582 ]
Mean: 71228.15653168518
Standard deviation: 3135.910014218122


ok lets do a random forest


In [14]:
from sklearn.ensemble import RandomForestRegressor

r_ForestRegressor=RandomForestRegressor()
r_ForestRegressor.fit(trainingDataPrepped,housing_labels)
rf_Predictions=r_ForestRegressor.predict(trainingDataPrepped)

rf_MSE=mean_squared_error(housing_labels,rf_Predictions)
rf_RMSE=np.sqrt(rf_MSE)
rf_RMSE


/Users/bernardroper/.conda/envs/ML-labs/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


22323.380310095537

So this is different from the book again...<br>
...but it is the same as the result from his project on GitHub, running on the same data
so I think we're good!<br>
Now some finishing touches. This is a Grid Search, in which the random forest is trained with a range of hyperparameters


In [15]:
from sklearn.model_selection import GridSearchCV

def grid_search(d_set,labels ,nEstimators=[10,20,30], maxFeatures=[ 4, 6, 8]):
    param_grid = [
    {'n_estimators': nEstimators, 'max_features': maxFeatures},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},]
    forest_reg = RandomForestRegressor()
    gridSearch = GridSearchCV(forest_reg, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
    gridSearch.fit(d_set,labels)
    return gridSearch
    
gridSearch=grid_search(trainingDataPrepped, housing_labels)

In [16]:
gridSearch.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=6, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=30,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [19]:
extraAttribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]

OneHotEncoder = full_pipe.named_transformers_["cat"]
cat1Hattribs = list(OneHotEncoder.categories_[0])
attributes = numeric + extraAttribs + cat1Hattribs
featureImportances = gridSearch.best_estimator_.feature_importances_
sorted(zip(featureImportances, attributes), reverse=True)

[(0.3527318789663025, 'median_income'),
 (0.1512008881097525, 'INLAND'),
 (0.10480044914037438, 'pop_per_hhold'),
 (0.0771736040029538, 'longitude'),
 (0.06876382939912976, 'bedrooms_per_room'),
 (0.06560081638596547, 'latitude'),
 (0.04964804980135733, 'rooms_per_hhold'),
 (0.04142972584135312, 'housing_median_age'),
 (0.017677673770851117, 'population'),
 (0.017622264330721317, 'total_rooms'),
 (0.016445346996838882, 'households'),
 (0.015370090929360656, 'total_bedrooms'),
 (0.009247666500152055, '<1H OCEAN'),
 (0.0063197012472474045, 'NEAR OCEAN'),
 (0.0058416870336896465, 'NEAR BAY'),
 (0.00012632754395020692, 'ISLAND')]

In [21]:
attributes
def stripCols(old_set):
    newTrainsetdf = pd.DataFrame(old_set, columns=attributes)
    return newTrainsetdf.drop(columns=["ISLAND","NEAR OCEAN","NEAR BAY","<1H OCEAN"], axis=1)

newTrainsetdfMKii = stripCols(trainingDataPrepped)

newTrainsetdfMKii


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_hhold,pop_per_hhold,bedrooms_per_room,INLAND
0,-1.156043,0.771950,0.743331,-0.493234,-0.445438,-0.636211,-0.420698,-0.614937,-0.312055,-0.086499,0.155318,0.0
1,-1.176025,0.659695,-1.165317,-0.908967,-1.036928,-0.998331,-1.022227,1.336459,0.217683,-0.033534,-0.836289,0.0
2,1.186849,-1.342183,0.186642,-0.313660,-0.153345,-0.433639,-0.093318,-0.532046,-0.465315,-0.092405,0.422200,0.0
3,-0.017068,0.313576,-0.290520,-0.362762,-0.396756,0.036041,-0.383436,-1.045566,-0.079661,0.089736,-0.196453,1.0
4,0.492474,-0.659299,-0.926736,1.856193,2.412211,2.724154,2.570975,-0.441437,-0.357834,-0.004194,0.269928,0.0
5,-0.696456,0.945009,-0.370047,0.143693,0.131447,0.025285,0.194138,-0.176435,-0.114867,-0.048003,-0.199264,1.0
6,0.537433,-0.748168,1.856709,-0.182253,-0.528198,-0.583328,-0.585720,2.366702,1.003599,-0.027646,-1.097923,0.0
7,1.166867,-0.696718,-0.210993,-0.272040,-0.586617,-0.433639,-0.324880,1.115239,0.000891,-0.051831,-1.073504,1.0
8,0.657325,-0.771554,1.459074,-0.354345,-0.167949,0.444770,-0.157197,-1.076906,-0.452724,0.110319,0.545225,0.0
9,0.647334,-0.757522,0.027588,-0.985660,-0.742400,1.138535,-0.745417,-1.772116,-1.174387,0.802788,3.534531,0.0


In [24]:
improvedRF_Reg =RandomForestRegressor(max_features=6,n_estimators=50)
improvedRF_Reg.fit(trainingDataPrepped,housing_labels)
improvedRF_RegPredicts=improvedRF_Reg.predict(trainingDataPrepped)

improvedRF_MSE=mean_squared_error(housing_labels,improvedRF_RegPredicts)
improvedRF_RMSE=np.sqrt(improvedRF_MSE)
improvedRF_RMSE

18792.68038879338

In [28]:
gs2 = grid_search(newTrainsetdfMKii,housing_labels,nEstimators=[30, 40,50,60], maxFeatures=[ 4, 6, 8,10,12])
gs2.best_estimator_


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=6, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)